In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
import matplotlib.pyplot as plt
from pprint import pprint

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

plt.style.use('ggplot')

%load_ext autoreload
%autoreload 2

In [3]:
from pyhocon import ConfigFactory

config = ConfigFactory.parse_file('../experiments/conf/nle_draft.hocon')
config

ConfigTree([('env',
             ConfigTree([('env_type', 'nle'),
                         ('video_path', 'artifacts/video/'),
                         ('goal_achieving_criterion', 'position'),
                         ('goal_type', 'from_current_episode'),
                         ('from_buffer_choice_params',
                          ConfigTree([('buffer_size', 100000),
                                      ('warmup_steps', 10)]))])),
            ('seed', 42),
            ('outputs',
             ConfigTree([('save_example', True),
                         ('path', 'artifacts/models/minigrid_worker.p')]))])

In [4]:
import sys
sys.path.append('../experiments')

from train_worker import gen_env, gen_navigation_env

In [5]:
env = gen_navigation_env(config['env'])
_ = env.reset()

In [6]:
for x in range(20):
    state, _, _, _ = env.step(1)

False
False
False
False
False
False
False
False
False
False
True
True
True
True
True
True
True
True
False
False


In [7]:
state.keys()

dict_keys(['state', 'goal_state'])

In [8]:
state['state']['position']

array([11,  7])

In [9]:
state['goal_state']['position']

array([13,  7])

In [23]:
env.last_observation

In [50]:
env.buffer_random_choice().keys()

dict_keys(['glyphs', 'chars', 'colors', 'specials', 'blstats', 'message', 'inv_glyphs', 'inv_strs', 'inv_letters', 'inv_oclasses', 'screen_descriptions', 'tty_chars', 'tty_colors', 'tty_cursor', 'position'])

In [24]:
import gym
import nle
env_ = gym.make("NetHackScore-v0", observation_keys=("glyphs", "blstats"))

In [25]:
state = env.reset()

In [27]:
print(env_.last_observation)

None


In [12]:
state.keys()

dict_keys(['state', 'goal_state'])

In [15]:
state['goal_state']

In [18]:
from collections import Counter

Counter(state['glyphs'].reshape(-1))

Counter({2359: 1594,
         2362: 1,
         2361: 17,
         2363: 1,
         2360: 5,
         2378: 33,
         2371: 2,
         2372: 1,
         155: 1,
         333: 1,
         397: 1,
         2364: 1,
         2365: 1})

In [23]:
glyphs = state["glyphs"]
T, B, *_ = glyphs.shape

blstats = state["blstats"]
blstats.view(T * B, -1).float()

ValueError: Type must be a sub-type of ndarray type

In [71]:
env.observation

AttributeError: 'NetHackScore' object has no attribute 'observation'

In [77]:
env.step_count

AttributeError: 'NetHackScore' object has no attribute 'step_count'

In [60]:
import gym
import nle
env = gym.make("NetHackScore-v0")
states0 = env.reset()  # each reset generates a new dungeon
env.render()


Hello Agent, welcome to NetHack!  You are a neutral male human Monk.            
                                                                                
                                                                                
      ------                                                                    
      |....|                                                                    
      |....|                                                                    
      |.d...                                                                    
      |.@..|                                                                    
      ---- -                                                                    
                                                                                
                                                                                
                                                                                
                           

In [61]:
states0['blstats'][:2]

array([8, 6])

In [62]:
states0['glyphs'][6, 8]

333

In [63]:
states1, _, _, _ = env.step(1)  # move agent '@' north
env.render()


You swap places with your little dog.                                           
                                                                                
                                                                                
      ------                                                                    
      |....|                                                                    
      |....|                      |                                             
      |.@...           ...........|                                             
      |.d..|             ----------                                             
      ----.-                                                                    
                                                                                
                                                                                
                                                                                
                           

In [64]:
states1['blstats'][:2]

array([8, 5])

In [66]:
states1['glyphs'][5, 8]

333